In [1]:
%load_ext autoreload
%autoreload 2
%load_ext jupyter_black

In [96]:
import numpy as np

import nzthermo as nzt
import nzthermo.functional as F
from nzthermo.core import ParcelProfile

pressure = np.array(
    [1013, 1000, 975, 950, 925, 900, 875, 850, 825, 800, 775, 750, 725, 700, 650, 600, 550, 500, 450, 400, 350, 300],
    dtype=np.float64,
)
pressure *= 100
temperature = np.array(
    [
        [243, 242, 241, 240, 239, 237, 236, 235, 233, 232, 231, 229, 228, 226, 235, 236, 234, 231, 226, 221, 217, 211],
        [250, 249, 248, 247, 246, 244, 243, 242, 240, 239, 238, 236, 235, 233, 240, 239, 236, 232, 227, 223, 217, 211],
        [293, 292, 290, 288, 287, 285, 284, 282, 281, 279, 279, 280, 279, 278, 275, 270, 268, 264, 260, 254, 246, 237],
        [300, 299, 297, 295, 293, 291, 292, 291, 291, 289, 288, 286, 285, 285, 281, 278, 273, 268, 264, 258, 251, 242],
    ],
    dtype=np.float64,
)
dewpoint = np.array(
    [
        [224, 224, 224, 224, 224, 223, 223, 223, 223, 222, 222, 222, 221, 221, 233, 233, 231, 228, 223, 218, 213, 207],
        [233, 233, 232, 232, 232, 232, 231, 231, 231, 231, 230, 230, 230, 229, 237, 236, 233, 229, 223, 219, 213, 207],
        [288, 288, 287, 286, 281, 280, 279, 277, 276, 275, 270, 258, 244, 247, 243, 254, 262, 248, 229, 232, 229, 224],
        [294, 294, 293, 292, 291, 289, 285, 282, 280, 280, 281, 281, 278, 274, 273, 269, 259, 246, 240, 241, 226, 219],
    ],
    dtype=np.float64,
)


def el(
    pressure: np.ndarray,
    temperature: np.ndarray,
    dewpoint: np.ndarray,
    parcel_profile: ParcelProfile | None = None,
    which="top",
):
    if parcel_profile is None:
        parcel_profile = nzt.parcel_profile(
            pressure, temperature[:, 0], dewpoint[:, 0], pressure[:1].repeat(temperature.shape[0])
        )
    pressure = parcel_profile.pressure
    temperature = parcel_profile.temperature
    N, Z = temperature.shape
    lcl_index = nx, zx = parcel_profile.lcl_index

    # [ dewpoint ]
    dewpoint = np.column_stack([dewpoint, np.full(N, np.nan)])
    Td = np.full_like(temperature, np.nan)
    mask = zx[:, None] >= np.arange(Z)
    nx, zx = np.nonzero(mask)
    Td[nx, zx - 1] = dewpoint[nx, zx]
    nx, zx = np.nonzero(~mask)
    Td[nx, zx] = dewpoint[nx, zx - 1]
    Td[lcl_index] = parcel_profile.lcl_temperature
    dewpoint = Td

    return F.intersect_nz(pressure, temperature, dewpoint, log_x=True)

    return dewpoint


el(pressure, temperature, dewpoint)

Intersection(x=array([72840.34476965, 72840.34476965, 62338.98811773,            nan,
       75973.87549461, 75973.87549461, 61225.9336347 ,            nan,
       94064.316261  , 94064.316261  , 92815.92020834, 92815.92020834]), y=array([221.1471183 , 221.1471183 , 233.        ,          nan,
       230.27213523, 230.27213523, 236.25269372,          nan,
       286.85549918, 286.85549918, 292.58698851, 292.58698851]), indices=array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 3, 3]))

In [59]:
arr = np.array([[1, 2, 3], [4, 5, 6]])

# Append a new value to each column
new_value = 7
np.column_stack((arr, [7, 7]))

array([[1, 2, 3, 7],
       [4, 5, 6, 7]])